In [ ]:
!pip install -q transformers einops accelerate scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.4 MB/s eta 0:00:00


In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from transformers import MambaConfig, MambaModel
from torch import nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from datetime import datetime
import joblib
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

✅ Using device: cuda


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Tesla.csv to Tesla.csv


In [ ]:
df = pd.read_csv('Tesla.csv')  # Or wherever your uploaded file is
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)

In [ ]:
current_day_features = [
    'day_of_week_sin', 'day_of_week_cos', 'day_of_year_sin', 'day_of_year_cos',
    'lag_return_1d', 'lag_return_2d', 'lag_return_3d', 'lag_return_4d', 'lag_return_5d'
]

prev_day_features = [
    'ema_12', 'ema_26', 'macd_line', 'macd_signal', 'rsi_14',
    'SPY_ret', 'QQQ_ret', 'VIX'
]

target_column = 'log_return'
date_column = 'Date'

# === 🧠 Build Input Features with Correct Temporal Alignment ===
features_aligned = []
dates_aligned = []
targets = []

for t in range(1, len(df)):
    cur_row = df.loc[t]
    prev_row = df.loc[t - 1]

    # Build aligned input vector
    feature_vector = list(cur_row[current_day_features]) + list(prev_row[prev_day_features])
    features_aligned.append(feature_vector)
    dates_aligned.append(cur_row[date_column])
    targets.append(cur_row[target_column])

# Convert to numpy
X_all = np.array(features_aligned)
y_all = np.array(targets)
dates_all = np.array(dates_aligned)

print(f"✅ Final shape: X={X_all.shape}, y={y_all.shape}")

✅ Final shape: X=(2501, 17), y=(2501,)


In [ ]:
from sklearn.preprocessing import StandardScaler
import joblib

# === 🎯 Only scale first 2000 inputs ===
X_base = X_all[:2000]

# === 🧪 Fit the scaler on X_base only ===
scaler = StandardScaler()
scaler.fit(X_base)

# === 💾 Save the scaler for later use ===
scaler_path = "/content/drive/MyDrive/Tesla_scaler.pkl"
joblib.dump(scaler, scaler_path)
print(f"✅ Scaler saved to: {scaler_path}")

✅ Scaler saved to: /content/drive/MyDrive/Tesla_scaler.pkl


In [ ]:
import torch
from torch import nn
from transformers import MambaConfig, MambaModel

# === ⚙️ Define the Mamba Regression Wrapper ===
class MambaRegressionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.config = MambaConfig(
            hidden_size=hidden_dim,
            num_hidden_layers=4,
            intermediate_size=4 * hidden_dim,
        )
        self.backbone = MambaModel(self.config)
        self.input_proj = nn.Linear(input_dim, hidden_dim)
        self.reg_head = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = self.input_proj(x)  # [batch, seq_len, hidden]
        outputs = self.backbone(inputs_embeds=x)
        last_hidden = outputs.last_hidden_state  # [batch, seq_len, hidden]
        return self.reg_head(last_hidden[:, -1, :])  # [batch, 1]

# === ✅ Instantiate model ===
input_dim = 17
hidden_dim = 64
seq_len = 90

model = MambaRegressionModel(input_dim, hidden_dim).to(device)

The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# === 🧱 Custom Dataset for Sliding Windows ===
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y, seq_len):
        self.X = X
        self.y = y
        self.seq_len = seq_len

    def __len__(self):
        return len(self.X) - self.seq_len

    def __getitem__(self, idx):
        x_seq = self.X[idx:idx + self.seq_len]  # [seq_len, feature_dim]
        y_label = self.y[idx + self.seq_len]    # next-day target
        return torch.tensor(x_seq, dtype=torch.float32), torch.tensor(y_label, dtype=torch.float32)

# === 🔧 Prepare Dataset + Dataloader ===
seq_len = 90
train_dataset = TimeSeriesDataset(X_base, y_all[:2000], seq_len)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# === ⚙️ Optimizer & Loss ===
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

# === 🔁 Training Loop ===
model.train()
epochs = 10

for epoch in range(epochs):
    total_loss = 0.0
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)  # [batch, seq_len, feature_dim]
        y_batch = y_batch.to(device).unsqueeze(1)  # [batch, 1]

        optimizer.zero_grad()
        preds = model(x_batch)  # [batch, 1]
        loss = loss_fn(preds, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"✅ Epoch {epoch+1}/{epochs} - Loss: {total_loss / len(train_loader):.6f}")

# === 💾 Save Model Weights and Config ===
from transformers import MambaConfig

# Create save directory in Google Drive
model_save_path = "/content/drive/MyDrive/Mamba_Base_Tesla_Checkpoint"
os.makedirs(model_save_path, exist_ok=True)

# Save model weights
torch.save(model.state_dict(), f"{model_save_path}/pytorch_model.bin")

# Save config (for reloading structure later)
model.config.save_pretrained(model_save_path)

print(f"✅ Base model saved to: {model_save_path}")

✅ Epoch 1/10 - Loss: 0.055442
✅ Epoch 2/10 - Loss: 0.004565
✅ Epoch 3/10 - Loss: 0.002617
✅ Epoch 4/10 - Loss: 0.002186
✅ Epoch 5/10 - Loss: 0.001911
✅ Epoch 6/10 - Loss: 0.001757
✅ Epoch 7/10 - Loss: 0.001660
✅ Epoch 8/10 - Loss: 0.001586
✅ Epoch 9/10 - Loss: 0.001514
✅ Epoch 10/10 - Loss: 0.001485
✅ Base model saved to: /content/drive/MyDrive/Mamba_Base_Tesla_Checkpoint


In [ ]:
# === 💾 Save Model Weights and Config ===
from transformers import MambaConfig

# Create save directory in Google Drive
model_save_path = "/content/drive/MyDrive/Mamba_Base_Tesla_Checkpoint"
os.makedirs(model_save_path, exist_ok=True)

# Save model weights
torch.save(model.state_dict(), f"{model_save_path}/pytorch_model.bin")

# Save config (for reloading structure later)
model.config.save_pretrained(model_save_path)

print(f"✅ Base model saved to: {model_save_path}")

✅ Base model saved to: /content/drive/MyDrive/Mamba_Base_Apple_Checkpoint


Rolling Prediction

In [ ]:
!pip install -q transformers einops accelerate scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.2 MB/s eta 0:00:00


In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from transformers import MambaConfig, MambaModel
from torch import nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from datetime import datetime
import joblib
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Tesla.csv to Tesla.csv


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# === 📄 Load Tesla.csv and Sort by Date ===
df = pd.read_csv('/content/Tesla.csv')  # Upload again or place in /content
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)

# === ✅ Define Feature Groups and Target ===
current_day_features = [
    'day_of_week_sin', 'day_of_week_cos', 'day_of_year_sin', 'day_of_year_cos',
    'lag_return_1d', 'lag_return_2d', 'lag_return_3d', 'lag_return_4d', 'lag_return_5d'
]
prev_day_features = [
    'ema_12', 'ema_26', 'macd_line', 'macd_signal', 'rsi_14',
    'SPY_ret', 'QQQ_ret', 'VIX'
]
target_column = 'log_return'
date_column = 'Date'

# === 🔄 Align Feature Matrix (Same as Training) ===
features_aligned, targets, dates_aligned = [], [], []
for t in range(1, len(df)):
    cur_row = df.loc[t]
    prev_row = df.loc[t - 1]
    features = list(cur_row[current_day_features]) + list(prev_row[prev_day_features])
    features_aligned.append(features)
    targets.append(cur_row[target_column])
    dates_aligned.append(cur_row[date_column])

X_all = np.array(features_aligned)
y_all = np.array(targets)
dates_all = np.array(dates_aligned)

print(f"✅ Aligned features: X_all shape = {X_all.shape}, Dates from {dates_all[0]} to {dates_all[-1]}")

✅ Aligned features: X_all shape = (2501, 17), Dates from 2015-01-26 00:00:00 to 2024-12-31 00:00:00


In [ ]:
# === 📈 Load the StandardScaler ===
scaler_path = "/content/drive/MyDrive/Tesla_scaler.pkl"
scaler = joblib.load(scaler_path)
print("✅ Scaler loaded")

✅ Scaler loaded


In [ ]:
# === 🧱 Define the Model Wrapper Again ===
class MambaRegressionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.config = MambaConfig.from_pretrained("/content/drive/MyDrive/Mamba_Base_Tesla_Checkpoint")
        self.backbone = MambaModel(self.config)
        self.input_proj = nn.Linear(input_dim, self.config.hidden_size)
        self.reg_head = nn.Linear(self.config.hidden_size, 1)

    def forward(self, x):
        x = self.input_proj(x)
        outputs = self.backbone(inputs_embeds=x)
        return self.reg_head(outputs.last_hidden_state[:, -1, :])  # Use final token

In [ ]:
# === 🔁 Load Saved Base Model ===
model = MambaRegressionModel(input_dim=17, hidden_dim=64).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/Mamba_Base_Tesla_Checkpoint/pytorch_model.bin"))
print("✅ Base model loaded")

The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.


✅ Base model loaded


In [ ]:
# === 🔧 Rolling Configs ===
seq_len = 90
window_size = 500
epochs_per_step = 2
learning_rate = 1e-5
output_csv = "/content/drive/MyDrive/Tesla_Mamba_Prediction.csv"

# === 🗃️ Resume from Last Saved Point (if exists) ===
if os.path.exists(output_csv):
    existing_df = pd.read_csv(output_csv)
    done_dates = set(pd.to_datetime(existing_df['Date']).dt.date)
    print(f"🔁 Resuming — already predicted {len(done_dates)} dates.")
else:
    existing_df = pd.DataFrame(columns=["Date", "Predicted_Log_Return"])
    done_dates = set()



🔁 Resuming — already predicted 225 dates.


In [ ]:
# === 🧱 Helper: Create sliding sequences per window ===
def generate_sequences(X, y, seq_len):
    sequences, labels = [], []
    for i in range(len(X) - seq_len):
        sequences.append(X[i:i+seq_len])
        labels.append(y[i+seq_len])
    return np.array(sequences), np.array(labels)

In [ ]:
# === 🔁 Rolling Loop ===
results = []
step = 0
start_date = pd.to_datetime("2023-01-03")

In [ ]:
for i in tqdm(range(len(dates_all))):
    cur_date = dates_all[i]

    # Skip if not enough data or already predicted
    if cur_date.date() < start_date.date():
        continue
    if cur_date.date() in done_dates:
        continue
    if i < window_size:
        continue

    # === Build rolling window ===
    X_window_raw = X_all[i - window_size:i]
    y_window_raw = y_all[i - window_size:i]
    X_scaled_window = scaler.transform(X_window_raw)

    # === Precompute sequences and labels for this window ===
    X_train_np, y_train_np = generate_sequences(X_scaled_window, y_window_raw, seq_len)
    X_train = torch.tensor(X_train_np, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train_np, dtype=torch.float32).unsqueeze(1).to(device)

    # === Reload base model and fine-tune ===
    model = MambaRegressionModel(input_dim=17, hidden_dim=64).to(device)
    model.load_state_dict(torch.load("/content/drive/MyDrive/Mamba_Base_Tesla_Checkpoint/pytorch_model.bin"))
    model.train()

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()

    for _ in range(epochs_per_step):
        perm = torch.randperm(len(X_train))
        for k in range(0, len(X_train), 32):
            idx = perm[k:k+32]
            x_batch = X_train[idx]
            y_batch = y_train[idx]

            optimizer.zero_grad()
            preds = model(x_batch)
            loss = loss_fn(preds, y_batch)
            loss.backward()
            optimizer.step()

    # === Predict next-day return ===
    X_today_scaled = scaler.transform([X_all[i]])
    X_today_tensor = torch.tensor(X_today_scaled, dtype=torch.float32).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        pred = model(X_today_tensor).cpu().item()

    results.append((cur_date.strftime("%Y-%m-%d"), pred))
    step += 1

    # === Save every 50 steps ===
    if step % 25 == 0 or i == len(dates_all) - 1:
        temp_df = pd.DataFrame(results, columns=["Date", "Tesla_Mamba"])
        combined_df = pd.concat([existing_df, temp_df], ignore_index=True)
        combined_df.to_csv(output_csv, index=False)
        print(f"💾 Saved {step} predictions to Drive")
        results = []
        existing_df = combined_df

    # === 🧹 Clean up model after prediction ===
    del model
    torch.cuda.empty_cache()

 90%|████████▉ | 2249/2501 [16:42<2:27:55, 35.22s/it]

💾 Saved 25 predictions to Drive


 91%|█████████ | 2274/2501 [33:16<2:30:18, 39.73s/it]

💾 Saved 50 predictions to Drive


 92%|█████████▏| 2299/2501 [49:50<2:13:41, 39.71s/it]

...

💾 Saved 250 predictions to Drive


100%|█████████▉| 2499/2501 [3:02:10<01:19, 39.68s/it]

💾 Saved 275 predictions to Drive


100%|██████████| 2501/2501 [3:03:30<00:00,  4.40s/it]

💾 Saved 277 predictions to Drive
